In [1]:
import pickle
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import json
import pandas as pd
import numpy as np
import os
import glob
import os.path
import chart_studio.plotly as py
import plotly.graph_objs as go
from itertools import product
from plotly.subplots import make_subplots
import plotly

In [2]:
naive_still_path = '/Users/huseyinelmas/Desktop/CCN-Lab/CCN-RSA/Results/Naive/Still/'
naive_video_path = '/Users/huseyinelmas/Desktop/CCN-Lab/CCN-RSA/Results/Naive/Video/'
prior_still_path = '/Users/huseyinelmas/Desktop/CCN-Lab/CCN-RSA/Results/Prior/Still/'
prior_video_path = '/Users/huseyinelmas/Desktop/CCN-Lab/CCN-RSA/Results/Prior/Video/'


In [3]:
naive_still_path = glob.glob(os.path.join(naive_still_path, '*.pkl'))[0]
naive_video_path = glob.glob(os.path.join(naive_video_path, '*.pkl'))[0]
prior_still_path = glob.glob(os.path.join(prior_still_path, '*.pkl'))[0]
prior_video_path = glob.glob(os.path.join(prior_video_path, '*.pkl'))[0]

In [6]:
#with open(path+electrode_region+'_results.pkl', 'rb') as f: 
#    electrode_models[electrode_region], significant_rdms, eeg_rdm_dist_metric, model_rdm_dist_metric, correct, alpha, el_r = pickle.load(f)
n_s_rdm_statistics_df, eeg_rdm_dist_metric, model_rdm_dist_metric = pd.read_pickle(naive_still_path)
n_v_rdm_statistics_df, eeg_rdm_dist_metric, model_rdm_dist_metric = pd.read_pickle(naive_video_path)
p_s_rdm_statistics_df, eeg_rdm_dist_metric, model_rdm_dist_metric = pd.read_pickle(prior_still_path)
p_v_rdm_statistics_df, eeg_rdm_dist_metric, model_rdm_dist_metric = pd.read_pickle(prior_video_path)

In [7]:
frames = [n_s_rdm_statistics_df, n_v_rdm_statistics_df, p_s_rdm_statistics_df,p_v_rdm_statistics_df]
rdm_df = pd.concat(frames)
rdm_df['Time window'] = rdm_df['Time window'].apply(lambda window: 2*(window[0]-100))

'/Users/huseyinelmas/Desktop/CCN-Lab/CCN-RSA/Results/Naive/Video/correlation_hamming_results.pkl'

In [ ]:
experiment_types = ['naive', 'prior']
stmuli_types = ['still', 'video']
combs = product(experiment_types,stmuli_types)
for exp_type, stm_type in combs:
    print(exp_type, stm_type)

In [8]:
def plot_experiment_stimuli_comparisons(electrode_region, 
                                        model_name):
    fig = make_subplots(rows=2, cols=2,shared_yaxes=False,
        subplot_titles=("Naive: Still vs Video", "Prior: Still vs Video", "Still: Naive vs Prior", "Video: Naive vs Prior"))
    
    colors = px.colors.qualitative.Plotly
    alpha=0.001
    correct = True
    n_test = 1
    if correct:
        n_test = 400    
    
    variables = ['still', 'video']
    constants = ['naive', 'prior']
    c_no = 0
    for col_no, constant in enumerate(constants):
        for variable in variables:
            df_to_plot = rdm_df[(rdm_df["Model name"] == model_name) 
                                & (rdm_df["Electrode Region"] == electrode_region)
                                & (rdm_df["Experiment Type"] == constant) 
                                & (rdm_df["Stimuli Type"] == variable)]
            pos_corr =  df_to_plot[ (df_to_plot["Kendall tau"] > 0 ) & ( df_to_plot["Kendall p-value"]/2 <= (alpha/n_test) )]["Time window"].tolist()
            fig.add_trace(go.Scatter(x=df_to_plot["Time window"],
                                    y=df_to_plot["Kendall tau"],
                                    legendgroup=constant + " " + variable,
                                    mode='lines',
                                    name=constant + " " + variable,
                                    line_color = colors[c_no]),
                                    row=1, col=col_no+1)
            fig.add_trace(
                        go.Scatter(
                            mode='markers',
                            legendgroup=constant + " " + variable,
                            name=constant + " " + variable,
                            x=pos_corr,
                            y=[-.1-(c_no*.02) for _ in range(len(pos_corr))],
                            marker=dict(
                                size=2,
                                color= colors[c_no],
                            )
                            ,
                            showlegend=False
                        ),
                        row = 1, col=col_no+1)
            c_no = c_no+1
    constants = ['still', 'video']
    variables = ['naive','prior']
    c_no = 0
    for col_no, constant in enumerate(constants):
        for  variable in variables:
            df_to_plot = rdm_df[(rdm_df["Model name"] == model_name) 
                                & (rdm_df["Electrode Region"] == electrode_region)
                                & (rdm_df["Experiment Type"] == variable) 
                                & (rdm_df["Stimuli Type"] == constant)]
            pos_corr =  df_to_plot[ (df_to_plot["Kendall tau"] > 0 ) & ( df_to_plot["Kendall p-value"]/2 <= (alpha/n_test) )]["Time window"].tolist()
            fig.add_trace(go.Scatter(x=df_to_plot["Time window"],
                                    y=df_to_plot["Kendall tau"],
                                    mode='lines',
                                    legendgroup=variable + " " + constant,
                                    name= variable + " " + constant,
                                    line_color = colors[c_no]),
                                    row=2, col=col_no+1)
            fig.add_trace(
                        go.Scatter(
                            mode='markers',
                            legendgroup=variable + " " + constant,
                            name=variable + " " + constant,
                            x=pos_corr,
                            y=[-.1-(c_no*.02) for _ in range(len(pos_corr))],
                            marker=dict(
                                size=2,
                                color= colors[c_no],
                            )
                            ,
                            showlegend=False
                        ),
                        row = 2, col=col_no+1)
            c_no = c_no+1
    fig.update_layout(height=1000, width=1000, title_text=electrode_region + " " + model_name)
    plotly.offline.plot(fig, filename = electrode_region + "_" + model_name +'_experiment_stimuli_comparison.html', auto_open=False)    

In [7]:
def plot_channels(model_name="biological_appearance_hamming"):
    colors = px.colors.qualitative.Plotly
    fig = make_subplots(rows=2, cols=2,shared_yaxes=True,
                        subplot_titles=("Naive Still ", "Naive Video", 
                                        "Prior Still", "Prior Video"))
    for row_no, exp_type in enumerate(['naive', 'prior']):
        for col_no, stm_type in enumerate(['still', 'video']):
                df_of_model = rdm_df[rdm_df["Model name"] == model_name]    
                df = df_of_model[(df_of_model["Experiment Type"] == exp_type) &
                                 (df_of_model["Stimuli Type"] == stm_type)]

                alpha=0.001
                correct = True
                n_test = 1
                if correct:
                    n_test = 400
                    sig_time = {}
                    pos_corr =  df[ (df["Kendall tau"] > 0 ) & ( df["Kendall p-value"]/2 <= (alpha/n_test) )]
                    for er in list(df["Electrode Region"].unique()):
                        sig_time[er] = pos_corr[pos_corr["Electrode Region"] ==  er]["Time window"].tolist()

                for i, er in enumerate(sig_time.keys()):
                    df_er = df[df["Electrode Region"]==er]
                    fig.add_trace(
                        go.Scatter(
                            mode='lines',
                            x=df_er["Time window"],
                            y=df_er["Kendall tau"], 
                            line_color=colors[i],
                            legendgroup=er,
                        name=er),
                        row = row_no+1, col=col_no+1
                    )
                    fig.add_trace(
                        go.Scatter(
                            mode='markers',
                            legendgroup=er,
                            name=er,
                            x=sig_time[er],
                            y=[-.1-(i*.02) for _ in range(len(sig_time[er]))],
                            marker=dict(
                                size=2,
                                color= colors[i],
                            )
                            ,
                            showlegend=False
                        ),
                        row = row_no+1, col=col_no+1
                    )
                    
    fig.update_layout(
        height=1000, width=1000,
        title= model_name)                
    plotly.offline.plot(fig, filename = model_name+'_channel_wise_comparison.html', auto_open=False)

In [8]:
for model_name in list(rdm_df["Model name"].unique()):
    plot_channels(model_name)

In [10]:
arg_dict = {"agent_hamming":["occipital"]
           }
for model, electrode_region_list in arg_dict.items():
    for electrode_region in electrode_region_list:    
        plot_experiment_stimuli_comparisons(electrode_region, model)



Experiment Type naive
Stimuli Type  still
Experiment Type naive
Stimuli Type  video
Experiment Type prior
Stimuli Type  still
Experiment Type prior
Stimuli Type  video
Experiment Type naive
Stimuli Type  still
Experiment Type prior
Stimuli Type  still
Experiment Type naive
Stimuli Type  video
Experiment Type prior
Stimuli Type  video


In [ ]:
trace_high = go.Scatter(
                x=df_to_plot[(df_to_plot["Experiment Type"] == "naive") & (df_to_plot["Stimuli Type"] == "still")]["Time window"],
                y=df_to_plot[(df_to_plot["Experiment Type"] == "naive") & (df_to_plot["Stimuli Type"] == "still")]["Kendall tau"],
                name = "Naive still",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

trace_low = go.Scatter(
                x=df_to_plot[(df_to_plot["Experiment Type"] == "naive") & (df_to_plot["Stimuli Type"] == "video")]["Time window"],
                y=df_to_plot[(df_to_plot["Experiment Type"] == "naive") & (df_to_plot["Stimuli Type"] == "video")]["Time window"],
                name = "Naive Video",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

data = [trace_high,trace_low]
py.iplot(data, filename = "Naive still vs Naive video")

In [ ]:
def plot_prep(df):
    df.columns = [c.replace(' ', '_') for c in df.columns]
    df['Time_window'] = df['Time_window'].apply(lambda window: 2*(window[0]-100))

    df.rename({'Time_window': 'time'}, axis=1, inplace=True)

In [ ]:
def load_models(path):
    electrode_regions = ['central', 'frontal', 'temporal', 'whole_brain', 'occipital', 'parietal', 'temporal']
    electrode_models = dict()
    #(exp_type, stimulus_type,electrode_region)

    for electrode_region in electrode_regions:
        f=path+electrode_region+'_results.pkl'
        #with open(path+electrode_region+'_results.pkl', 'rb') as f: 
        #    electrode_models[electrode_region], significant_rdms, eeg_rdm_dist_metric, model_rdm_dist_metric, correct, alpha, el_r = pickle.load(f)
        electrode_models[electrode_region], significant_rdms, eeg_rdm_dist_metric, model_rdm_dist_metric, correct, alpha, el_r = pd.read_pickle(f)
        plot_prep(electrode_models[electrode_region])

    return electrode_models
    

In [ ]:
def plot(electrode_models, electrode_region):
    df= electrode_models[electrode_region]
    fig = px.line(df, x=df.time, y=df.Kendall_tau, 
                color="Model_name")
    shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0="-200",
                y0=0,
                x1="0",
                y1=1,
                fillcolor="cornflowerblue",
                opacity=0.3,
                layer="below",
                line_width=0,
            )
    ]

    '''updatemenus=[
        dict(
            direction = "down",
            buttons=[ {
                    'method': 'restyle',
                    'label': electrode_region,
                    'args': [ 
                        {'y': [ electrode_models[electrode_region].Kendall_tau ]},
                    ]
                } for electrode_region in electrode_regions],
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.4,
            xanchor="left",
            y=1.13,
            yanchor="top"
        ),
    ]'''


    fig.update_layout(
        #updatemenus=updatemenus,
        shapes=shapes,
        #annotations=[ dict(text="Select the electrode: ",  showarrow=False,
        #                         x=0, y=1.08, yref="paper", align="left"),
        #],
        title="Electrode: " + electrode_region
    )

    fig.show()

In [ ]:
pth = '/home/sena/PycharmProjects/CCN-RSA/RSA_Results/Naive/Still/'
electrode_models=load_models(pth)
plot(electrode_models, 'parietal' )

In [ ]:
paths = {}
for exp_type in ['Naive', 'Prior']:
    for stm_type in ['Still', 'Video']:
        paths[(exp_type,stm_type)] ='/home/sena/PycharmProjects/CCN-RSA/RSA_Results/'+exp_type+'/'+stm_type+'/'


In [ ]:
electrode_models=load_models(paths['Prior','Video'])

In [ ]:
test_df = electrode_models['occipital']

In [ ]:
pos_corr_rdms =  test_df[ test_df.Kendall_tau > 0]

alpha=0.001
correct = True
n_test = 1
if correct:
    n_test = 400
    significant_rdms = pos_corr_rdms[pos_corr_rdms["Kendall_p-value"]/2 <= (alpha/n_test)]

In [ ]:
mark = [t for t in significant_rdms[significant_rdms["Model_name"] == "movement_nature_hamming"].time ]

In [ ]:
mark

In [ ]:
df = test_df[test_df.Model_name == "movement_nature_hamming"]
fig = px.line(df, x=df.time, y=df.Kendall_tau, 
            color="Model_name")
shapes=[
    dict(
        type="rect",
        # x-reference is assigned to the x-values
        xref="x",

        # y-reference is assigned to the plot paper [0,1]
        yref="paper",
        x0="-200",
        y0=0,
        x1="0",
        y1=1,
        fillcolor="cornflowerblue",
        opacity=0.3,
        layer="below",
        line_width=0
    )
]

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=mark,
        y=[-.1 for i in range(len(mark))],
        marker=dict(
            color='LightSkyBlue',
            size=5
        ),
        showlegend=False
    )
)

fig.update_layout(
    shapes=shapes,
    title="Testing markers " 
)

fig.show()

In [ ]:
plot(electrode_models, 'occipital' )

In [ ]:
electrode_models=load_models(paths['Naive','Video'])
plot(electrode_models, 'occipital' )

In [ ]:
df = electrode_models['parietal']

In [ ]:
fig = px.line(df, x=df.time, y=df.Kendall_tau, 
            color="Model_name")
shapes=[
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0="-200",
            y0=0,
            x1="0",
            y1=1,
            fillcolor="cornflowerblue",
            opacity=0.3,
            layer="below",
            line_width=0,
        )
]

'''updatemenus=[
    dict(
        type = "buttons",
        direction = "left",
        buttons=list([
            dict(
                args=["type", "models"],
                label="Models",
                method="restyle"
            ),
            dict(
                args=["type", "electrode_site"],
                label="Electrode site",
                method="restyle"
            ),
            dict(
                args=["type", "experiment_type"],
                label="Experiment type",
                method="restyle"
            ),
            dict(
                args=["type", "stimulus_type"],
                label="Stimulus type",
                method="restyle"
            )
        ]),
        pad={"r": 10, "t": 10},
        showactive=True,
        xanchor="left",
        x=0.31,
        yanchor="top",
        y=1.125
    ),
    dict(
        #type = "buttons",
        direction = "down",
        buttons=list([
            dict(
                args=["type", "frontal"],
                label="Frontal",
                method="restyle"
            ),
            dict(
                args=["type", "central"],
                label="Central",
                method="restyle"
            ),

            dict(
                args=["type", "temporal"],
                label="Temporal",
                method="restyle"
            ),
            dict(
                args=["type", "parietal"],
                label="Parietal",
                method="restyle"
            ),
            dict(
                args=["type", "occipital"],
                label="Occipital",
                method="restyle"
            ),
            dict(
                args=["type", "whole_brain"],
                label="Whole brain",
                method="restyle"
            )
        ]),
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.4,
        xanchor="left",
        y=1.02,
        yanchor="top"
    ),
        dict(
        direction = "down",
        buttons=list([
            dict(
                args=["type", "naive"],
                label="Naive",
                method="restyle"
            ),
            dict(
                args=["type", "prior"],
                label="Prior",
                method="restyle"
            )
        ]),
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.6,
        xanchor="left",
        y=1.02,
        yanchor="top"
    ),
        dict(
        #type = "buttons",
        direction = "down",
        buttons=list([
            dict(
                args=["type", "video"],
                label="Video",
                method="restyle"
            ),
            dict(
                args=["type", "still"],
                label="Still",
                method="restyle"
            )
        ]),
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.8,
        xanchor="left",
        y=1.02,
        yanchor="top"
    )
]
'''

fig.update_layout(
    #updatemenus=updatemenus,
    shapes=shapes,
    annotations=[ dict(text="Compare models",  showarrow=False,
                             x=0, y=1.08, yref="paper", align="left")
    ])





fig.show()

In [ ]:
model_correlation = dict()

for model_name in df.Model_name.unique():
    model_correlation[model_name]  = df.loc[df.Model_name==model_name].Kendall_tau

In [ ]:
type(model_correlation[model_name])

In [ ]:


fig = px.line(df, x=df.time, y=df.Kendall_tau, 
            color="Model_name")
shapes=[
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0="-200",
            y0=0,
            x1="0",
            y1=1,
            fillcolor="cornflowerblue",
            opacity=0.3,
            layer="below",
            line_width=0,
        )
]

updatemenus=[
    dict(
        type = "buttons",
        direction = "left",
        buttons=list([
            dict(
                args=["type", "models"],
                label="Models",
                method="restyle"
            ),
            dict(
                args=["type", "electrode_site"],
                label="Electrode site",
                method="restyle"
            ),
            dict(
                args=["type", "experiment_type"],
                label="Experiment type",
                method="restyle"
            ),
            dict(
                args=["type", "stimulus_type"],
                label="Stimulus type",
                method="restyle"
            )
        ]),
        pad={"r": 10, "t": 10},
        showactive=True,
        xanchor="left",
        x=0.31,
        yanchor="top",
        y=1.125
    ),
    dict(
        #type = "buttons",
        direction = "down",
        buttons=list([
            dict(
                args=["frontal", electrode_models["frontal"][["Model_name","Kendall_tau"]]],
                label="Frontal",
                method="restyle"
            ),
            dict(
                args=["central", electrode_models["central"][["Model_name","Kendall_tau"]]],
                label="Central",
                method="restyle"
            ),

            dict(
                args=["type", "temporal"],
                label="Temporal",
                method="restyle"
            ),
            dict(
                args=["type", "parietal"],
                label="Parietal",
                method="restyle"
            ),
            dict(
                args=["type", "occipital"],
                label="Occipital",
                method="restyle"
            ),
            dict(
                args=["type", "whole_brain"],
                label="Whole brain",
                method="restyle"
            )
        ]),
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.4,
        xanchor="left",
        y=1.02,
        yanchor="top"
    ),
        dict(
        direction = "down",
        buttons=list([
            dict(
                args=["type", "naive"],
                label="Naive",
                method="restyle"
            ),
            dict(
                args=["type", "prior"],
                label="Prior",
                method="restyle"
            )
        ]),
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.6,
        xanchor="left",
        y=1.02,
        yanchor="top"
    ),
        dict(
        #type = "buttons",
        direction = "down",
        buttons=list([
            dict(
                args=["type", "video"],
                label="Video",
                method="restyle"
            ),
            dict(
                args=["type", "still"],
                label="Still",
                method="restyle"
            )
        ]),
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.8,
        xanchor="left",
        y=1.02,
        yanchor="top"
    )
]


fig.update_layout(
    updatemenus=updatemenus,
    shapes=shapes,
    annotations=[ dict(text="Compare by: ",  showarrow=False,
                             x=0, y=1.08, yref="paper", align="left")
    ])





fig.show()

In [ ]:
significant_rdms.columns = [c.replace(' ', '_') for c in significant_rdms.columns]
significant_rdms['Time_window'] = significant_rdms['Time_window'].apply(lambda window: window[0]-100)

significant_rdms.rename({'Time_window': 'time'}, axis=1, inplace=True)

In [ ]:
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=significant_rdms,
        y=significant_rdms.time.unique()))

In [ ]:
import pandas as pd

In [ ]:
significant_rdms

In [ ]:
import plotly
fig = dict(
layout = dict(
    xaxis1 = {'domain': [0.0, 0.44], 'anchor': 'y1', 'title': '1', 'range': [-2.25, 3.25]},
    yaxis1 = {'domain': [0.0, 1.0], 'anchor': 'x1', 'title': 'y', 'range': [-1, 11]},
    xaxis2 = {'domain': [0.56, 1.0], 'anchor': 'y2', 'title': '2', 'range': [-2.25, 3.25]},
    yaxis2 = {'domain': [0.0, 1.0], 'anchor': 'x2', 'title': 'y', 'range': [-1, 11]},
    title  = '',
    margin = {'t': 50, 'b': 50, 'l': 50, 'r': 50},
    updatemenus = [{'buttons': [{'args': [['0', '1', '2', '3'], {'frame': {'duration': 500.0, 'redraw': False}, 'fromcurrent': True, 'transition': {'duration': 0, 'easing': 'linear'}}], 'label': 'Play', 'method': 'animate'}, {'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate', 'transition': {'duration': 0}}], 'label': 'Pause', 'method': 'animate'}], 'direction': 'left', 'pad': {'r': 10, 't': 85}, 'showactive': True, 'type': 'buttons', 'x': 0.1, 'y': 0, 'xanchor': 'right', 'yanchor': 'top'}],
    sliders = [{'yanchor': 'top', 'xanchor': 'left', 'currentvalue': {'font': {'size': 16}, 'prefix': 'Frame: ', 'visible': True, 'xanchor': 'right'}, 'transition': {'duration': 500.0, 'easing': 'linear'}, 'pad': {'b': 10, 't': 50}, 'len': 0.9, 'x': 0.1, 'y': 0, 
                'steps': [{'args': [['0'], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': False}, 'transition': {'duration': 0, 'easing': 'linear'}}], 'label': '0', 'method': 'animate'}, 
                          {'args': [['1'], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': False}, 'transition': {'duration': 0, 'easing': 'linear'}}], 'label': '1', 'method': 'animate'}, 
                          {'args': [['2'], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': False}, 'transition': {'duration': 0, 'easing': 'linear'}}], 'label': '2', 'method': 'animate'},
                          {'args': [['3'], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': False}, 'transition': {'duration': 0, 'easing': 'linear'}}], 'label': '3', 'method': 'animate'}, 
                ]}]
),
data = [
    {'type': 'scatter', 'name': 'f1', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  4,   1,   1, 1,   4,   9], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(255,79,38,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(255,79,38,0.600000)', 'legendgroup': 'f1', 'showlegend': True, 'xaxis': 'x1', 'yaxis': 'y1'},
{'type': 'scatter', 'name': 'f2', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  2.5,   1,   1, 1,   2.5,   1], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(79,102,165,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(79,102,165,0.600000)', 'legendgroup': 'f2', 'showlegend': True, 'xaxis': 'x2', 'yaxis': 'y2'},
],
frames = [
    {'name' : '0', 'layout' : {},
     'data': [
         {'type': 'scatter', 'name': 'f1', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  5,   8,   3, 2,   4,   0], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(255,79,38,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(255,79,38,0.600000)', 'legendgroup': 'f1', 'showlegend': True, 'xaxis': 'x1', 'yaxis': 'y1'}, 
#              {'type': 'scatter', 'name': 'f1', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  4,   7,   2, 1,   3,   0], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(255,79,38,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(255,79,38,0.600000)', 'legendgroup': 'f1', 'showlegend': True, 'xaxis': 'x1', 'yaxis': 'y1'},
         {'type': 'scatter', 'name': 'f2', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  3,   7,   4, 8,   5,   9], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(79,102,165,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(79,102,165,0.600000)', 'legendgroup': 'f2', 'showlegend': True, 'xaxis': 'x2', 'yaxis': 'y2'},
#              {'type': 'scatter', 'name': 'f2', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  2,   8,   3, 9,   4,   9], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(79,102,165,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(79,102,165,0.600000)', 'legendgroup': 'f2', 'showlegend': True, 'xaxis': 'x2', 'yaxis': 'y2'}
     ],
    },
    {'name' : '1', 'layout' : {},
     'data': [
         {'type': 'scatter', 'name': 'f1', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  4,   1,   1, 1,   4,   9], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(255,79,38,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(255,79,38,0.600000)', 'legendgroup': 'f1', 'showlegend': True, 'xaxis': 'x1', 'yaxis': 'y1'}, 
         {'type': 'scatter', 'name': 'f2', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  2.5,   1,   1, 1,   2.5,   1], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(79,102,165,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(79,102,165,0.600000)', 'legendgroup': 'f2', 'showlegend': True, 'xaxis': 'x2', 'yaxis': 'y2'}],
    },
    {'name' : '2', 'layout' : {},
     'data': [
         {'type': 'scatter', 'name': 'f1', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  5,   8,   3, 2,   4,   0], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(255,79,38,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(255,79,38,0.600000)', 'legendgroup': 'f1', 'showlegend': True, 'xaxis': 'x1', 'yaxis': 'y1'}, 
         {'type': 'scatter', 'name': 'f2', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  3,   7,   4, 8,   5,   9], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(79,102,165,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(79,102,165,0.600000)', 'legendgroup': 'f2', 'showlegend': True, 'xaxis': 'x2', 'yaxis': 'y2'}],
    },
    {'name' : '3', 'layout' : {},
     'data': [
         {'type': 'scatter', 'name': 'f1', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  4,   1,   1, 1,   4,   9], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(255,79,38,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(255,79,38,0.600000)', 'legendgroup': 'f1', 'showlegend': True, 'xaxis': 'x1', 'yaxis': 'y1'}, 
         {'type': 'scatter', 'name': 'f2', 'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 'y': [  2.5,   1,   1, 1,   2.5,   1], 'hoverinfo': 'name+text', 'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 'line': {'color': 'rgba(79,102,165,1.000000)'}, 'mode': 'markers+lines', 'fillcolor': 'rgba(79,102,165,0.600000)', 'legendgroup': 'f2', 'showlegend': True, 'xaxis': 'x2', 'yaxis': 'y2'}],
    }
]
)
plotly.offline.plot(fig, auto_open=True)